In [1]:
import random
random.seed(42)

import torch
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [2]:
import numpy as np
import pandas as pd
import shap
import tqdm

from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import GroupShuffleSplit
import sklearn.metrics as skmetrics
from sklearn.calibration import CalibrationDisplay
from sklearn.utils import resample

from skopt import BayesSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

from process_fe import create_feature_engineering_datasets
from data import data_to_array_dict, get_data_date_split, get_data_date_id_split, get_feature_colnames
from utils import stratification
from plotting import paper_theme, ReliabilityDisplay, ShapDisplay, risk_feature_plot
import metrics
from shap_calculator import calc_shap_df

from tqdm_style import tqdm_style
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold, ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score

/Users/kx/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Fixing the training set and predicting on the last year

In [22]:
N_DAYS = 3
THRESHOLDS = [0.3, 0.8]
DATES_SPLIT = {
    "date_train_start": "2021-06-28",
    "date_train_end": "2023-01-01",
    "date_test_end": "2024-01-01",
}

In [23]:
col_nice_names = {
    'awake_freq': 'Night time Awake Frequency', 
    'bathroom_daytime_freq': 'Daytime Bathroom Frequency', 
    'bathroom_daytime_freq_ma': 'Daytime Bathroom Frequency MA', 
    'bathroom_daytime_freq_ma_delta': 'Daytime Bathroom Frequency MA Delta', 
    'bathroom_freq': 'Bathroom Frequency',
    'bathroom_nighttime_freq': 'Night time Bathroom Frequency', 
    'bathroom_nighttime_freq_ma': 'Night time Bathroom Frequency MA', 
    'bathroom_nighttime_freq_ma_delta': 'Night time Bathroom Frequency MA Delta', 
    'bathroom_relative_transition_time_delta_mean': 'Mean Relative Bathroom Transition Time Delta',
    'bathroom_relative_transition_time_delta_std': 'STD Relative Bathroom Transition Time Delta',
    'bedroom_freq': 'Bedroom Frequency',
    'daily_entropy': 'Daily Entropy', 
    'hallway_freq': 'Hallway Frequency', 
    'heart_rate_mean': 'Mean Night Time Heart Rate',
    'heart_rate_std': 'STD Night Time Heart Rate', 
    'kitchen_freq': 'Kitchen Frequency', 
    'lounge_freq': 'Lounge Frequency', 
    'previous_uti': 'Number of Previous UTIs',
    'respiratory_rate_mean': 'Mean Night Time Respiratory Rate', 
    'respiratory_rate_std': 'STD Night Time Respiratory Rate',
}

## A MLP

In [24]:
fe_data = create_feature_engineering_datasets(reload=False)

In [25]:
print(fe_data.columns)

Index(['patient_id', 'date', 'bathroom_freq', 'bedroom_freq', 'hallway_freq',
       'kitchen_freq', 'lounge_freq', 'awake_freq', 'heart_rate_mean',
       'heart_rate_std', 'respiratory_rate_mean', 'respiratory_rate_std',
       'bathroom_nighttime_freq', 'bathroom_nighttime_freq_ma',
       'bathroom_nighttime_freq_ma_delta', 'bathroom_daytime_freq',
       'bathroom_daytime_freq_ma', 'bathroom_daytime_freq_ma_delta',
       'daily_entropy', 'bathroom_relative_transition_time_delta_mean',
       'bathroom_relative_transition_time_delta_std', 'previous_uti'],
      dtype='object')


In [26]:

data_train, data_test, _ = get_data_date_split(
    fe_data, dates_split=DATES_SPLIT, n_days=N_DAYS, impute=True
)


X_train, y_train, ids_train, sample_weight = (
    data_train['X'], data_train['y'], data_train["id"], data_train['sample_weight']
)

X_test, y_test, ids_test, dates_test = (
    data_test['X'], data_test['y'], data_test["id"], data_test['date']
)

/Users/kx/MSc/uti_risk_to_share/process_fe.py:353: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels[


In [27]:
# Convert IDs to sets
set_ids_train = set(ids_train)
set_ids_test = set(ids_test)

# Check for intersection
common_ids = set_ids_train.intersection(set_ids_test)

# Check if there are any common elements
if common_ids:
    print(f"There are {len(common_ids)} common IDs between train and test datasets.")
else:
    print("IDs in train and test datasets are unique to each other.")

There are 41 common IDs between train and test datasets.


In [28]:
# List of IDs to remove
ids_to_remove = ['AboZyUBeiQW3nVCcbXGpay', 'NZjrVTZQR1w9LPJMt26MbG', 'XVb8nztyc2LYPCAewZq11S', 'XdbAAiDw1vd3Bjbo9EVo1B']

# Create a boolean index where False indicates IDs that need to be removed
indices_to_keep = ~np.isin(data_test['id'], ids_to_remove)

# Use this index to filter all related arrays in data_test
data_test['X'] = data_test['X'][indices_to_keep]
data_test['y'] = data_test['y'][indices_to_keep]
data_test['id'] = data_test['id'][indices_to_keep]
data_test['date'] = data_test['date'][indices_to_keep] if 'date' in data_test else None

X_test, y_test, ids_test, dates_test = (
    data_test['X'], data_test['y'], data_test["id"], data_test['date']
)

In [29]:
print("id_train:", ids_train.shape)
print("id_test:", ids_test.shape)

id_train: (1839,)
id_test: (1145,)


In [30]:
# Convert the NumPy array to a pandas Series
ids_train_series = pd.Series(ids_train)

# Now you can use the nunique() method
unique_count = ids_train_series.nunique()

print("Number of unique elements in ids_train:", unique_count)

Number of unique elements in ids_train: 69


In [31]:
ids_train_series = pd.Series(ids_test)

# Now you can use the nunique() method
unique_count = ids_train_series.nunique()

print("Number of unique elements in ids_test:", unique_count)

Number of unique elements in ids_test: 51


In [32]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (1839, 3, 20)
Shape of y_train: (1839,)


In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import numpy as np

# Define flatten function
def flatten(x):
    return x.reshape(x.shape[0], -1)

# Apply flattening
X_train_flattened = flatten(X_train)
X_test_flattened = flatten(X_test)

# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flattened)
X_test_scaled = scaler.transform(X_test_flattened)

# Convert to PyTorch tensors
X_train_torch = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)
sample_weight_torch = torch.tensor(sample_weight, dtype=torch.float32)

In [34]:
X_test_torch = torch.tensor(X_test_scaled, dtype=torch.float32)

y_test_torch = torch.tensor(y_test, dtype=torch.float32)

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import numpy as np

In [41]:
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

In [37]:
gender_map['gender_encoded'] = gender_map['Gender PwD'].map({'Male': 0, 'Female': 1})


patient_gender_dict = dict(zip(gender_map['patient_id'], gender_map['gender_encoded']))

gender_train = [patient_gender_dict.get(patient_id, 0) for patient_id in ids_train] 

gender_test = [patient_gender_dict.get(patient_id, 0) for patient_id in ids_test] 

gender_train = np.array(gender_train).astype(int)
gender_test = np.array(gender_test).astype(int)

In [38]:
# Create a mapping for 'Does PwD live alone?'
carer_map = {'No': 0, 'Yes': 1, 'Care home': 2}

# Map 'Does PwD live alone?' to numeric values
gender_map['carer_encoded'] = gender_map['Does PwD live alone?'].map(carer_map)

# Create carer_dict mapping patient_id to carer_encoded
carer_dict = dict(zip(gender_map['patient_id'], gender_map['carer_encoded']))

# Generate carer_train and carer_test
carer_train = [carer_dict.get(patient_id, 0) for patient_id in ids_train]
carer_test = [carer_dict.get(patient_id, 0) for patient_id in ids_test]

carer_train = np.array(carer_train).astype(int)
carer_test = np.array(carer_test).astype(int)

/var/folders/4g/hwd_s7k56xx1xf7d3wkh1kfm0000gn/T/ipykernel_14428/820093588.py:14: RuntimeWarning: invalid value encountered in cast
  carer_train = np.array(carer_train).astype(int)


In [39]:
print("Gender distribution in training data:", np.unique(gender_train, return_counts=True))
print("Gender distribution in testing data:", np.unique(gender_test, return_counts=True))

Gender distribution in training data: (array([0, 1]), array([1183,  656]))
Gender distribution in testing data: (array([0, 1]), array([695, 450]))


In [40]:
from torch.utils.data import Dataset
import pickle

class CustomDataset(Dataset):
    def __init__(self, features, labels, gender, carer_status, patient_id, sample_weight=None):
        """
        Constructor for the dataset.
        :param features: The input features (numpy array).
        :param labels: The labels corresponding to the features (numpy array).
        :param loss_labels: The loss labels corresponding to the features (numpy array).
        :param gender: The gender information (numpy array).
        :param carer_status: The carer status information (numpy array).
        :param patient_id: The patient ID information (numpy array).
        :param sample_weight: Optional sample weights (numpy array).
        """
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

        self.gender = torch.tensor(gender, dtype=torch.long)  # Assuming gender is categorical
        self.carer_status = torch.tensor(carer_status, dtype=torch.long)  # Assuming carer status is categorical
        self.patient_id = patient_id
        
        if sample_weight is not None:
            self.sample_weight = torch.tensor(sample_weight, dtype=torch.float32)
        else:
            self.sample_weight = None

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        if self.sample_weight is not None:
            return (self.features[index], self.labels[index], 
                    self.gender[index], self.carer_status[index], self.sample_weight[index], 
                    self.patient_id[index])
        return (self.features[index], self.labels[index], 
                self.gender[index], self.carer_status[index], self.patient_id[index])


In [41]:

# Instantiate datasets
train_dataset = CustomDataset(X_train_scaled, y_train, gender_train, carer_train, ids_train, sample_weight)
test_dataset = CustomDataset(X_test_scaled, y_test, gender_test, carer_test, ids_test, sample_weight)

In [43]:
print(X_train_scaled.shape)

(1839, 60)


# Simple

In [43]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_rate):
        super(MLP, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.layer3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = self.layer3(x)
        return x


In [44]:
# Define the hyperparameter grid
hyperparameter_grid = {'lr': [0.001, 0.005, 0.01], 'dropout_rate': [0, 0.2, 0.5]}

In [45]:
# Custom training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for features, labels, gender, carer_status, patient_id, sample_weight in train_loader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validate the model
        model.eval()
        val_loss = 0
        all_labels = []
        all_preds = []
        all_genders = []
        with torch.no_grad():
            for features, labels, gender, carer_status, patient_i, sample_weight in val_loader:
                outputs = model(features)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())
                all_genders.extend(gender.cpu().numpy())
        
        val_loss /= len(val_loader)
        all_labels = np.array(all_labels)
        all_preds = np.array(all_preds)
        all_genders = np.array(all_genders)

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=0)
        sensitivity = recall_score(all_labels, all_preds, average='binary', zero_division=0)

        # Calculate metrics for males and females separately
        male_indices = all_genders == 0
        female_indices = all_genders == 1

        male_accuracy = accuracy_score(all_labels[male_indices], all_preds[male_indices])
        male_precision = precision_score(all_labels[male_indices], all_preds[male_indices], average='binary', zero_division=0)
        male_sensitivity = recall_score(all_labels[male_indices], all_preds[male_indices], average='binary', zero_division=0)

        female_accuracy = accuracy_score(all_labels[female_indices], all_preds[female_indices])
        female_precision = precision_score(all_labels[female_indices], all_preds[female_indices], average='binary', zero_division=0)
        female_sensitivity = recall_score(all_labels[female_indices], all_preds[female_indices], average='binary', zero_division=0)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}, '
              f'Overall - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Sensitivity: {sensitivity:.4f}, '
              f'Male - Accuracy: {male_accuracy:.4f}, Precision: {male_precision:.4f}, Sensitivity: {male_sensitivity:.4f}, '
              f'Female - Accuracy: {female_accuracy:.4f}, Precision: {female_precision:.4f}, Sensitivity: {female_sensitivity:.4f}')
    
    return val_loss, accuracy, precision, sensitivity, male_accuracy, male_precision, male_sensitivity, female_accuracy, female_precision, female_sensitivity

In [46]:
# Cross-validation with hyperparameter tuning
best_hyperparams = None
best_val_loss = float('inf')
best_metrics = None

skf = StratifiedKFold(n_splits=10)
for params in ParameterGrid(hyperparameter_grid):
    fold_metrics = {
        'accuracy': [], 'precision': [], 'sensitivity': [],
        'male_accuracy': [], 'male_precision': [], 'male_sensitivity': [],
        'female_accuracy': [], 'female_precision': [], 'female_sensitivity': []
    }
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        print(f'Fold {fold+1} with params: {params}')
        train_subset = Subset(train_dataset, train_index)
        val_subset = Subset(train_dataset, val_index)
        train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)
        model = MLP(input_size=X_train_scaled.shape[1], hidden_size1=30, hidden_size2=10, output_size=2, dropout_rate=params['dropout_rate'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=params['lr'])
        val_loss, accuracy, precision, sensitivity, male_accuracy, male_precision, male_sensitivity, female_accuracy, female_precision, female_sensitivity = train_model(model, train_loader, val_loader, criterion, optimizer)
        
        fold_metrics['accuracy'].append(accuracy)
        fold_metrics['precision'].append(precision)
        fold_metrics['sensitivity'].append(sensitivity)
        fold_metrics['male_accuracy'].append(male_accuracy)
        fold_metrics['male_precision'].append(male_precision)
        fold_metrics['male_sensitivity'].append(male_sensitivity)
        fold_metrics['female_accuracy'].append(female_accuracy)
        fold_metrics['female_precision'].append(female_precision)
        fold_metrics['female_sensitivity'].append(female_sensitivity)
    
    avg_val_loss = np.mean([val_loss])
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_hyperparams = params
        best_metrics = fold_metrics

# Print the best hyperparameters and corresponding metrics
print("Best Hyperparameters:", best_hyperparams)
for metric in best_metrics:
    print(f'{metric}: Mean = {np.mean(best_metrics[metric]):.4f}, Std = {np.std(best_metrics[metric]):.4f}')

Fold 1 with params: {'dropout_rate': 0, 'lr': 0.001}
Epoch 1/10, Loss: 0.6875, Overall - Accuracy: 0.6848, Precision: 0.0000, Sensitivity: 0.0000, Male - Accuracy: 0.7119, Precision: 0.0000, Sensitivity: 0.0000, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 2/10, Loss: 0.6988, Overall - Accuracy: 0.6793, Precision: 0.0000, Sensitivity: 0.0000, Male - Accuracy: 0.7062, Precision: 0.0000, Sensitivity: 0.0000, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 3/10, Loss: 0.7247, Overall - Accuracy: 0.6848, Precision: 0.5000, Sensitivity: 0.0172, Male - Accuracy: 0.7119, Precision: 0.5000, Sensitivity: 0.0196, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 4/10, Loss: 0.7600, Overall - Accuracy: 0.6739, Precision: 0.3750, Sensitivity: 0.0517, Male - Accuracy: 0.7006, Precision: 0.3750, Sensitivity: 0.0588, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 5/10, Loss: 0.7830, Overall - Accuracy: 0

In [47]:
# Function to perform bootstrap sampling
def bootstrap_sample(dataset, patient_ids, proportion=0.8):
    sampled_indices = []
    for pid in np.unique(patient_ids):
        pid_indices = np.where(patient_ids == pid)[0]
        sample_size = int(proportion * len(pid_indices))
        sampled_pid_indices = np.random.choice(pid_indices, size=sample_size, replace=True)
        sampled_indices.extend(sampled_pid_indices)
    return Subset(dataset, sampled_indices)

In [48]:
# Function to evaluate the model
def evaluate_model(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    all_genders = []
    with torch.no_grad():
        for features, labels, gender, carer_status, patient_id, sample_weight in data_loader:
            outputs = model(features)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            all_genders.extend(gender.cpu().numpy())

    total_loss /= len(data_loader)
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    all_genders = np.array(all_genders)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    sensitivity = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    male_indices = all_genders == 0
    female_indices = all_genders == 1

    male_accuracy = accuracy_score(all_labels[male_indices], all_preds[male_indices])
    male_precision = precision_score(all_labels[male_indices], all_preds[male_indices], average='macro', zero_division=0)
    male_sensitivity = recall_score(all_labels[male_indices], all_preds[male_indices], average='macro', zero_division=0)

    female_accuracy = accuracy_score(all_labels[female_indices], all_preds[female_indices])
    female_precision = precision_score(all_labels[female_indices], all_preds[female_indices], average='macro', zero_division=0)
    female_sensitivity = recall_score(all_labels[female_indices], all_preds[female_indices], average='macro', zero_division=0)

    return total_loss, accuracy, precision, sensitivity, male_accuracy, male_precision, male_sensitivity, female_accuracy, female_precision, female_sensitivity

# Bootstrap sampling and training the best model
num_bootstrap_samples = 5
bootstrap_results = {
    'accuracy': [], 'precision': [], 'sensitivity': [],
    'male_accuracy': [], 'male_precision': [], 'male_sensitivity': [],
    'female_accuracy': [], 'female_precision': [], 'female_sensitivity': []
}

for _ in range(num_bootstrap_samples):
    # Bootstrap sampling 80% of data points for each patient
    bootstrap_subset = bootstrap_sample(train_dataset, ids_train, proportion=0.8)
    bootstrap_loader = DataLoader(bootstrap_subset, batch_size=128, shuffle=True)
    
    model = MLP(input_size=X_train_scaled.shape[1], hidden_size1=30, hidden_size2=10, output_size=2, dropout_rate=best_hyperparams['dropout_rate'])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_hyperparams['lr'])
    
    # Train with the best hyperparameters
    train_model(model, bootstrap_loader, DataLoader(test_dataset, batch_size=128, shuffle=False), criterion, optimizer)
    
    # Evaluate on the full test dataset
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
    results = evaluate_model(model, test_loader, criterion)
    
    bootstrap_results['accuracy'].append(results[1])
    bootstrap_results['precision'].append(results[2])
    bootstrap_results['sensitivity'].append(results[3])
    bootstrap_results['male_accuracy'].append(results[4])
    bootstrap_results['male_precision'].append(results[5])
    bootstrap_results['male_sensitivity'].append(results[6])
    bootstrap_results['female_accuracy'].append(results[7])
    bootstrap_results['female_precision'].append(results[8])
    bootstrap_results['female_sensitivity'].append(results[9])

# Print bootstrap results
for metric in bootstrap_results:
    print(f'{metric.capitalize()}: Mean = {np.mean(bootstrap_results[metric]):.4f}, Std = {np.std(bootstrap_results[metric]):.4f}')

Epoch 1/10, Loss: 0.7303, Overall - Accuracy: 0.2061, Precision: 0.1923, Sensitivity: 0.8240, Male - Accuracy: 0.1640, Precision: 0.1553, Sensitivity: 0.9130, Female - Accuracy: 0.2711, Precision: 0.2557, Sensitivity: 0.7481
Epoch 2/10, Loss: 0.6932, Overall - Accuracy: 0.4926, Precision: 0.2318, Sensitivity: 0.5720, Male - Accuracy: 0.5137, Precision: 0.2233, Sensitivity: 0.7826, Female - Accuracy: 0.4600, Precision: 0.2477, Sensitivity: 0.3926
Epoch 3/10, Loss: 0.6469, Overall - Accuracy: 0.7598, Precision: 0.4352, Sensitivity: 0.3360, Male - Accuracy: 0.8288, Precision: 0.4859, Sensitivity: 0.6000, Female - Accuracy: 0.6533, Precision: 0.2941, Sensitivity: 0.1111
Epoch 4/10, Loss: 0.5939, Overall - Accuracy: 0.8122, Precision: 0.6923, Sensitivity: 0.2520, Male - Accuracy: 0.8978, Precision: 0.7750, Sensitivity: 0.5391, Female - Accuracy: 0.6800, Precision: 0.0909, Sensitivity: 0.0074
Epoch 5/10, Loss: 0.5504, Overall - Accuracy: 0.8114, Precision: 0.7073, Sensitivity: 0.2320, Male -

## Concatenate demographics

In [43]:
# Define the MLP model with concatenation
class concat_MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_rate):
        super(concat_MLP, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1 + 1, hidden_size2)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.layer3 = nn.Linear(hidden_size2 + 1, output_size)

    def forward(self, x, gender):
        x = self.layer1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = torch.cat((x, gender.unsqueeze(1)), dim=1)
        x = self.layer2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = torch.cat((x, gender.unsqueeze(1)), dim=1)
        x = self.layer3(x)
        return x

In [44]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for features, labels, gender, carer_status, patient_id, sample_weight in train_loader:
            optimizer.zero_grad()
            outputs = model(features, gender)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validate the model
        model.eval()
        val_loss = 0
        all_labels = []
        all_preds = []
        all_genders = []
        with torch.no_grad():
            for features, labels, gender, carer_status, patient_id, sample_weight in val_loader:
                outputs = model(features, gender)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())
                all_genders.extend(gender.cpu().numpy())
        
        val_loss /= len(val_loader)
        all_labels = np.array(all_labels)
        all_preds = np.array(all_preds)
        all_genders = np.array(all_genders)

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=0)
        sensitivity = recall_score(all_labels, all_preds, average='binary', zero_division=0)

        # Calculate metrics for males and females separately
        male_indices = all_genders == 0
        female_indices = all_genders == 1

        male_accuracy = accuracy_score(all_labels[male_indices], all_preds[male_indices])
        male_precision = precision_score(all_labels[male_indices], all_preds[male_indices], average='binary', zero_division=0)
        male_sensitivity = recall_score(all_labels[male_indices], all_preds[male_indices], average='binary', zero_division=0)

        female_accuracy = accuracy_score(all_labels[female_indices], all_preds[female_indices])
        female_precision = precision_score(all_labels[female_indices], all_preds[female_indices], average='binary', zero_division=0)
        female_sensitivity = recall_score(all_labels[female_indices], all_preds[female_indices], average='binary', zero_division=0)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}, '
              f'Overall - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Sensitivity: {sensitivity:.4f}, '
              f'Male - Accuracy: {male_accuracy:.4f}, Precision: {male_precision:.4f}, Sensitivity: {male_sensitivity:.4f}, '
              f'Female - Accuracy: {female_accuracy:.4f}, Precision: {female_precision:.4f}, Sensitivity: {female_sensitivity:.4f}')
    
    return val_loss, accuracy, precision, sensitivity, male_accuracy, male_precision, male_sensitivity, female_accuracy, female_precision, female_sensitivity

In [45]:
# Hyperparameter grid
hyperparameter_grid = {'lr': [0.001, 0.005, 0.01], 'dropout_rate': [0, 0.2, 0.5]}

# Cross-validation
best_hyperparams = None
best_val_loss = float('inf')
best_metrics = None

skf = StratifiedKFold(n_splits=10)
for params in ParameterGrid(hyperparameter_grid):
    fold_metrics = {
        'accuracy': [], 'precision': [], 'sensitivity': [],
        'male_accuracy': [], 'male_precision': [], 'male_sensitivity': [],
        'female_accuracy': [], 'female_precision': [], 'female_sensitivity': []
    }
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        print(f'Fold {fold+1} with params: {params}')
        train_subset = Subset(train_dataset, train_index)
        val_subset = Subset(train_dataset, val_index)
        train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)
        model = concat_MLP(input_size=X_train_scaled.shape[1], hidden_size1=30, hidden_size2=10, output_size=2, dropout_rate=params['dropout_rate'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=params['lr'])
        val_loss, accuracy, precision, sensitivity, male_accuracy, male_precision, male_sensitivity, female_accuracy, female_precision, female_sensitivity = train_model(model, train_loader, val_loader, criterion, optimizer)
        
        fold_metrics['accuracy'].append(accuracy)
        fold_metrics['precision'].append(precision)
        fold_metrics['sensitivity'].append(sensitivity)
        fold_metrics['male_accuracy'].append(male_accuracy)
        fold_metrics['male_precision'].append(male_precision)
        fold_metrics['male_sensitivity'].append(male_sensitivity)
        fold_metrics['female_accuracy'].append(female_accuracy)
        fold_metrics['female_precision'].append(female_precision)
        fold_metrics['female_sensitivity'].append(female_sensitivity)
    
    avg_val_loss = np.mean([val_loss])
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_hyperparams = params
        best_metrics = fold_metrics

# Print the best hyperparameters and corresponding metrics
print("Best Hyperparameters:", best_hyperparams)
for metric in best_metrics:
    print(f'{metric}: Mean = {np.mean(best_metrics[metric]):.4f}, Std = {np.std(best_metrics[metric]):.4f}')

Fold 1 with params: {'dropout_rate': 0, 'lr': 0.001}
Epoch 1/10, Loss: 0.6905, Overall - Accuracy: 0.4457, Precision: 0.2885, Sensitivity: 0.5172, Male - Accuracy: 0.4633, Precision: 0.2885, Sensitivity: 0.5882, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 2/10, Loss: 0.6754, Overall - Accuracy: 0.6902, Precision: 0.5294, Sensitivity: 0.1552, Male - Accuracy: 0.7175, Precision: 0.5294, Sensitivity: 0.1765, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 3/10, Loss: 0.6696, Overall - Accuracy: 0.6957, Precision: 0.5714, Sensitivity: 0.1379, Male - Accuracy: 0.7232, Precision: 0.5714, Sensitivity: 0.1569, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 4/10, Loss: 0.6725, Overall - Accuracy: 0.6793, Precision: 0.4444, Sensitivity: 0.0690, Male - Accuracy: 0.7062, Precision: 0.4444, Sensitivity: 0.0784, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 5/10, Loss: 0.6882, Overall - Accuracy: 0

In [46]:
# Function to perform bootstrap sampling
def bootstrap_sample(dataset, patient_ids, proportion=0.8):
    sampled_indices = []
    for pid in np.unique(patient_ids):
        pid_indices = np.where(patient_ids == pid)[0]
        sample_size = int(proportion * len(pid_indices))
        sampled_pid_indices = np.random.choice(pid_indices, size=sample_size, replace=True)
        sampled_indices.extend(sampled_pid_indices)
    return Subset(dataset, sampled_indices)

# Function to evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    all_genders = []
    with torch.no_grad():
        for inputs, labels, gender, carer_status, sample_weight, patient_id in dataloader:
            outputs = model(inputs, gender)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_genders.extend(gender.cpu().numpy())

    # Overall metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='binary', zero_division=0)  # Sensitivity
    f1 = f1_score(all_labels, all_preds, average='binary', zero_division=0)

    # Gender-specific metrics
    males = [i for i, g in enumerate(all_genders) if g == 0]
    females = [i for i, g in enumerate(all_genders) if g == 1]

    male_labels = [all_labels[i] for i in males]
    male_preds = [all_preds[i] for i in males]
    female_labels = [all_labels[i] for i in females]
    female_preds = [all_preds[i] for i in females]

    male_accuracy = accuracy_score(male_labels, male_preds)
    male_precision = precision_score(male_labels, male_preds, zero_division=0)
    male_recall = recall_score(male_labels, male_preds, zero_division=0)  # Sensitivity
    male_f1 = f1_score(male_labels, male_preds, zero_division=0)

    female_accuracy = accuracy_score(female_labels, female_preds)
    female_precision = precision_score(female_labels, female_preds, zero_division=0)
    female_recall = recall_score(female_labels, female_preds, zero_division=0)  # Sensitivity
    female_f1 = f1_score(female_labels, female_preds, zero_division=0)

    return {
        'overall': {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
        },
        'male': {
            'accuracy': male_accuracy,
            'precision': male_precision,
            'recall': male_recall,
            'f1': male_f1
        },
        'female': {
            'accuracy': female_accuracy,
            'precision': female_precision,
            'recall': female_recall,
            'f1': female_f1
        }
    }

# Bootstrap sampling and training the best model
num_bootstrap_samples = 5
bootstrap_results = {
    'accuracy': [], 'precision': [], 'sensitivity': [],
    'male_accuracy': [], 'male_precision': [], 'male_sensitivity': [],
    'female_accuracy': [], 'female_precision': [], 'female_sensitivity': []
}

for _ in range(num_bootstrap_samples):
    # Bootstrap sampling 80% of data points for each patient
    bootstrap_subset = bootstrap_sample(train_dataset, ids_train, proportion=0.8)
    bootstrap_loader = DataLoader(bootstrap_subset, batch_size=128, shuffle=True)
    
    model = concat_MLP(input_size=X_train_scaled.shape[1], hidden_size1=30, hidden_size2=10, output_size=2, dropout_rate=best_hyperparams['dropout_rate'])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_hyperparams['lr'])
    
    # Train with the best hyperparameters
    train_model(model, bootstrap_loader, DataLoader(test_dataset, batch_size=128, shuffle=False), criterion, optimizer)
    
    # Evaluate on the full test dataset
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
    results = evaluate_model(model, test_loader)
    
    bootstrap_results['accuracy'].append(results['overall']['accuracy'])
    bootstrap_results['precision'].append(results['overall']['precision'])
    bootstrap_results['sensitivity'].append(results['overall']['recall'])
    bootstrap_results['male_accuracy'].append(results['male']['accuracy'])
    bootstrap_results['male_precision'].append(results['male']['precision'])
    bootstrap_results['male_sensitivity'].append(results['male']['recall'])
    bootstrap_results['female_accuracy'].append(results['female']['accuracy'])
    bootstrap_results['female_precision'].append(results['female']['precision'])
    bootstrap_results['female_sensitivity'].append(results['female']['recall'])

# Print bootstrap results
for metric in bootstrap_results:
    print(f'{metric.capitalize()}: Mean = {np.mean(bootstrap_results[metric]):.4f}, Std = {np.std(bootstrap_results[metric]):.4f}')

Epoch 1/10, Loss: 0.6182, Overall - Accuracy: 0.7930, Precision: 0.8824, Sensitivity: 0.0600, Male - Accuracy: 0.8532, Precision: 0.8824, Sensitivity: 0.1304, Female - Accuracy: 0.7000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 2/10, Loss: 0.5893, Overall - Accuracy: 0.7921, Precision: 0.8333, Sensitivity: 0.0600, Male - Accuracy: 0.8518, Precision: 0.8333, Sensitivity: 0.1304, Female - Accuracy: 0.7000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 3/10, Loss: 0.5638, Overall - Accuracy: 0.7939, Precision: 0.7188, Sensitivity: 0.0920, Male - Accuracy: 0.8547, Precision: 0.7188, Sensitivity: 0.2000, Female - Accuracy: 0.7000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 4/10, Loss: 0.5365, Overall - Accuracy: 0.8000, Precision: 0.7234, Sensitivity: 0.1360, Male - Accuracy: 0.8647, Precision: 0.7234, Sensitivity: 0.2957, Female - Accuracy: 0.7000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 5/10, Loss: 0.5109, Overall - Accuracy: 0.8087, Precision: 0.7627, Sensitivity: 0.1800, Male -

## Source Embedding

In [44]:
# Define the MLP model with embedding and dropout
class MLPWithEmbedding(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, embedding_dim=4, dropout_rate=0):
        super(MLPWithEmbedding, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=2, embedding_dim=embedding_dim)
        self.layer1 = nn.Linear(input_size + embedding_dim, hidden_size1)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.layer3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x, gender):
        gender_embedded = self.embedding(gender)
        x = torch.cat((x, gender_embedded), dim=1)
        x = self.layer1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = self.layer3(x)
        return x

In [45]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for features, labels, gender, carer_status, patient_id, sample_weight in train_loader:
            optimizer.zero_grad()
            outputs = model(features, gender)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validate the model
        model.eval()
        val_loss = 0
        all_labels = []
        all_preds = []
        all_genders = []
        with torch.no_grad():
            for features, labels, gender, carer_status, patient_id, sample_weight in val_loader:
                outputs = model(features, gender)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())
                all_genders.extend(gender.cpu().numpy())
        
        val_loss /= len(val_loader)
        all_labels = np.array(all_labels)
        all_preds = np.array(all_preds)
        all_genders = np.array(all_genders)

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='binary', zero_division=0)
        sensitivity = recall_score(all_labels, all_preds, average='binary', zero_division=0)

        # Calculate metrics for males and females separately
        male_indices = all_genders == 0
        female_indices = all_genders == 1

        male_accuracy = accuracy_score(all_labels[male_indices], all_preds[male_indices])
        male_precision = precision_score(all_labels[male_indices], all_preds[male_indices], average='binary', zero_division=0)
        male_sensitivity = recall_score(all_labels[male_indices], all_preds[male_indices], average='binary', zero_division=0)

        female_accuracy = accuracy_score(all_labels[female_indices], all_preds[female_indices])
        female_precision = precision_score(all_labels[female_indices], all_preds[female_indices], average='binary', zero_division=0)
        female_sensitivity = recall_score(all_labels[female_indices], all_preds[female_indices], average='binary', zero_division=0)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss:.4f}, '
              f'Overall - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Sensitivity: {sensitivity:.4f}, '
              f'Male - Accuracy: {male_accuracy:.4f}, Precision: {male_precision:.4f}, Sensitivity: {male_sensitivity:.4f}, '
              f'Female - Accuracy: {female_accuracy:.4f}, Precision: {female_precision:.4f}, Sensitivity: {female_sensitivity:.4f}')
    
    return val_loss, accuracy, precision, sensitivity, male_accuracy, male_precision, male_sensitivity, female_accuracy, female_precision, female_sensitivity

In [46]:
# Hyperparameter grid
hyperparameter_grid = {'lr': [0.001, 0.005, 0.01], 'dropout_rate': [0, 0.2, 0.5]}

# Cross-validation with hyperparameter tuning
best_hyperparams = None
best_val_loss = float('inf')
best_metrics = None

skf = StratifiedKFold(n_splits=10)
for params in ParameterGrid(hyperparameter_grid):
    fold_metrics = {
        'accuracy': [], 'precision': [], 'sensitivity': [],
        'male_accuracy': [], 'male_precision': [], 'male_sensitivity': [],
        'female_accuracy': [], 'female_precision': [], 'female_sensitivity': []
    }
    for fold, (train_index, val_index) in enumerate(skf.split(X_train_scaled, y_train)):
        print(f'Fold {fold+1} with params: {params}')
        train_subset = Subset(train_dataset, train_index)
        val_subset = Subset(train_dataset, val_index)
        train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)
        model = MLPWithEmbedding(input_size=X_train_scaled.shape[1], hidden_size1=30, hidden_size2=10, output_size=2, embedding_dim=4, dropout_rate=params['dropout_rate'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=params['lr'])
        val_loss, accuracy, precision, sensitivity, male_accuracy, male_precision, male_sensitivity, female_accuracy, female_precision, female_sensitivity = train_model(model, train_loader, val_loader, criterion, optimizer)
        
        fold_metrics['accuracy'].append(accuracy)
        fold_metrics['precision'].append(precision)
        fold_metrics['sensitivity'].append(sensitivity)
        fold_metrics['male_accuracy'].append(male_accuracy)
        fold_metrics['male_precision'].append(male_precision)
        fold_metrics['male_sensitivity'].append(male_sensitivity)
        fold_metrics['female_accuracy'].append(female_accuracy)
        fold_metrics['female_precision'].append(female_precision)
        fold_metrics['female_sensitivity'].append(female_sensitivity)
    
    avg_val_loss = np.mean([val_loss])
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_hyperparams = params
        best_metrics = fold_metrics

# Print the best hyperparameters and corresponding metrics
print("Best Hyperparameters:", best_hyperparams)
for metric in best_metrics:
    print(f'{metric}: Mean = {np.mean(best_metrics[metric]):.4f}, Std = {np.std(best_metrics[metric]):.4f}')

Fold 1 with params: {'dropout_rate': 0, 'lr': 0.001}
Epoch 1/10, Loss: 0.6760, Overall - Accuracy: 0.6848, Precision: 0.5000, Sensitivity: 0.0345, Male - Accuracy: 0.7119, Precision: 0.5000, Sensitivity: 0.0392, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 2/10, Loss: 0.6725, Overall - Accuracy: 0.6902, Precision: 0.6667, Sensitivity: 0.0345, Male - Accuracy: 0.7175, Precision: 0.6667, Sensitivity: 0.0392, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 3/10, Loss: 0.6712, Overall - Accuracy: 0.6739, Precision: 0.3333, Sensitivity: 0.0345, Male - Accuracy: 0.7006, Precision: 0.3333, Sensitivity: 0.0392, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 4/10, Loss: 0.6718, Overall - Accuracy: 0.6630, Precision: 0.2500, Sensitivity: 0.0345, Male - Accuracy: 0.6893, Precision: 0.2500, Sensitivity: 0.0392, Female - Accuracy: 0.0000, Precision: 0.0000, Sensitivity: 0.0000
Epoch 5/10, Loss: 0.6773, Overall - Accuracy: 0

In [47]:
# Function to perform bootstrap sampling
def bootstrap_sample(dataset, patient_ids, proportion=0.8):
    sampled_indices = []
    for pid in np.unique(patient_ids):
        pid_indices = np.where(patient_ids == pid)[0]
        sample_size = int(proportion * len(pid_indices))
        sampled_pid_indices = np.random.choice(pid_indices, size=sample_size, replace=True)
        sampled_indices.extend(sampled_pid_indices)
    return Subset(dataset, sampled_indices)

# Function to evaluate the model
def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    all_genders = []
    with torch.no_grad():
        for features, labels, gender, carer_status, patient_id, sample_weight in dataloader:
            outputs = model(features, gender)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_genders.extend(gender.cpu().numpy())

    # Overall metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='binary', zero_division=0)  # Sensitivity
    f1 = f1_score(all_labels, all_preds, average='binary', zero_division=0)

    # Gender-specific metrics
    males = [i for i, g in enumerate(all_genders) if g == 0]
    females = [i for i, g in enumerate(all_genders) if g == 1]

    male_labels = [all_labels[i] for i in males]
    male_preds = [all_preds[i] for i in males]
    female_labels = [all_labels[i] for i in females]
    female_preds = [all_preds[i] for i in females]

    male_accuracy = accuracy_score(male_labels, male_preds)
    male_precision = precision_score(male_labels, male_preds, average='binary', zero_division=0)
    male_recall = recall_score(male_labels, male_preds, average='binary', zero_division=0)  # Sensitivity
    male_f1 = f1_score(male_labels, male_preds, average='binary', zero_division=0)

    female_accuracy = accuracy_score(female_labels, female_preds)
    female_precision = precision_score(female_labels, female_preds, average='binary', zero_division=0)
    female_recall = recall_score(female_labels, female_preds, average='binary', zero_division=0)  # Sensitivity
    female_f1 = f1_score(female_labels, female_preds, average='binary', zero_division=0)

    return {
        'overall': {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        },
        'male': {
            'accuracy': male_accuracy,
            'precision': male_precision,
            'recall': male_recall,
            'f1': male_f1
        },
        'female': {
            'accuracy': female_accuracy,
            'precision': female_precision,
            'recall': female_recall,
            'f1': female_f1
        }
    }

# Bootstrap sampling and training the best model
num_bootstrap_samples = 5
bootstrap_results = {
    'accuracy': [], 'precision': [], 'sensitivity': [],
    'male_accuracy': [], 'male_precision': [], 'male_sensitivity': [],
    'female_accuracy': [], 'female_precision': [], 'female_sensitivity': []
}

for _ in range(num_bootstrap_samples):
    # Bootstrap sampling 80% of data points for each participant
    bootstrap_subset = bootstrap_sample(train_dataset, ids_train, proportion=0.8)
    bootstrap_loader = DataLoader(bootstrap_subset, batch_size=128, shuffle=True)
    
    model = MLPWithEmbedding(input_size=X_train_scaled.shape[1], hidden_size1=30, hidden_size2=10, output_size=2, embedding_dim=4, dropout_rate=best_hyperparams['dropout_rate'])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_hyperparams['lr'])
    
    # Train with the best hyperparameters
    train_model(model, bootstrap_loader, DataLoader(test_dataset, batch_size=128, shuffle=False), criterion, optimizer)
    
    # Evaluate on the full test dataset
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
    results = evaluate_model(model, test_loader)
    
    bootstrap_results['accuracy'].append(results['overall']['accuracy'])
    bootstrap_results['precision'].append(results['overall']['precision'])
    bootstrap_results['sensitivity'].append(results['overall']['recall'])
    bootstrap_results['male_accuracy'].append(results['male']['accuracy'])
    bootstrap_results['male_precision'].append(results['male']['precision'])
    bootstrap_results['male_sensitivity'].append(results['male']['recall'])
    bootstrap_results['female_accuracy'].append(results['female']['accuracy'])
    bootstrap_results['female_precision'].append(results['female']['precision'])
    bootstrap_results['female_sensitivity'].append(results['female']['recall'])

# Print bootstrap results
for metric in bootstrap_results:
    print(f'{metric.capitalize()}: Mean = {np.mean(bootstrap_results[metric]):.4f}, Std = {np.std(bootstrap_results[metric]):.4f}')

Epoch 1/10, Loss: 0.7248, Overall - Accuracy: 0.3214, Precision: 0.2419, Sensitivity: 0.9880, Male - Accuracy: 0.3338, Precision: 0.1990, Sensitivity: 1.0000, Female - Accuracy: 0.3022, Precision: 0.2980, Sensitivity: 0.9778
Epoch 2/10, Loss: 0.6566, Overall - Accuracy: 0.6550, Precision: 0.3650, Sensitivity: 0.7840, Male - Accuracy: 0.7957, Precision: 0.4366, Sensitivity: 0.8087, Female - Accuracy: 0.4378, Precision: 0.3179, Sensitivity: 0.7630
Epoch 3/10, Loss: 0.5871, Overall - Accuracy: 0.8035, Precision: 0.5604, Sensitivity: 0.4640, Male - Accuracy: 0.9022, Precision: 0.7423, Sensitivity: 0.6261, Female - Accuracy: 0.6511, Precision: 0.4000, Sensitivity: 0.3259
Epoch 4/10, Loss: 0.5251, Overall - Accuracy: 0.8175, Precision: 0.6881, Sensitivity: 0.3000, Male - Accuracy: 0.8993, Precision: 0.8082, Sensitivity: 0.5130, Female - Accuracy: 0.6911, Precision: 0.4444, Sensitivity: 0.1185
Epoch 5/10, Loss: 0.4713, Overall - Accuracy: 0.8166, Precision: 0.7083, Sensitivity: 0.2720, Male -